In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()


[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: C:\Users\anime\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'google.colab'

In [3]:
! pip install pandas numpy matplotlib seaborn scikit-learn statsmodels xgboost lightgbm pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00


In [4]:
import pandas as pd

In [5]:
!kaggle datasets download -d rohitsahoo/sales-forecasting

Dataset URL: https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting
License(s): GPL-2.0
  0% 0.00/480k [00:00<?, ?B/s]
100% 480k/480k [00:00<00:00, 87.0MB/s]


In [6]:
!unzip sales-forecasting.zip

Archive:  sales-forecasting.zip
  inflating: train.csv               


In [7]:
df = pd.read_csv('train.csv')

In [8]:
df.drop(columns=['Row ID', 'Ship Mode', 'Customer ID', 'Country','City', 'Category', 'Sub-Category', 'Product Name'], inplace=True)

In [9]:
df = df.dropna(axis=0)

In [25]:
df['order_date']= pd.to_datetime(df['Order Date'], dayfirst=True)
df['ship_date']= pd.to_datetime(df['Ship Date'], dayfirst=True)
df['year_month']= df['order_date'].apply(lambda x: x.strftime("%Y-%m"))
df['year']= df['order_date'].dt.year
df['month']=df['order_date'].dt.month_name()
df['ship_in_days']= (df['ship_date']- df['order_date']).dt.days

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

In [16]:
numeric_columns = df.select_dtypes(include=['number']).columns
rf_sales = pd.DataFrame(df.groupby(by=['order_date'])[numeric_columns].sum())

In [32]:
x = rf_sales.drop('Sales', axis=1)
y = rf_sales['Sales']

In [43]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(x_train, y_train)

preds = rf_model.predict(x_test)

rmse_rf = sqrt(mean_squared_error(y_test, preds))
mae_rf = mean_absolute_error(y_test, preds)
r2_rf = r2_score(y_test, preds)

print("Root Mean Squared Error for Random Forest:", rmse_rf)

Root Mean Squared Error for Random Forest: 1582.2284482109728
